Install and import pyathena package

In [ ]:
!pip install pyathena

In [4]:
from pyathena import connect
import pandas as pd

In [32]:
with open("../src/main/resources/application.conf", "r") as ins:
    config = dict([line.replace("\n", '').replace('spark.', '').split("=") for line in ins])

In [33]:
conn_athena = connect(aws_access_key_id=config['awskey'],
           aws_secret_access_key=config['awssecret'],
           s3_staging_dir='s3://capstonedend/',
           region_name="eu-west-1")

Create all Amazon Athena tables

In [3]:
!python create_athenadb.py

Root buckets is: s3://capstonedend/
Athena database name is: capstone
Athena logs output path is: s3://capstonedend/athena_logs/
Creating sources table
Table sources has been created
Creating reporters table
Table reporters has been created
Creating fire_names table
Table fire_names has been created
Creating fire_causes table
Table fire_causes has been created
Creating fire_sizes table
Table fire_sizes has been created
Creating owners table
Table owners has been created
Creating locations table
Table locations has been created
Creating dates table
Table dates has been created
Creating weather_types table
Table weather_types has been created
Creating fires_fact table
Table fires_fact has been created
Creating weather_fact table
Table weather_fact has been created


### Data Quality check

In [34]:
pd.read_sql("""
SELECT 'sources' AS table_name, count(*) AS count FROM capstone.sources where source_id IS NULL
    UNION
SELECT 'reporters' AS table_name, count(*) as count FROM capstone.reporters where reporter_id IS NULL
    UNION
SELECT 'fire_causes' AS table_name, count(*) as count FROM capstone.fire_causes where fire_cause_id IS NULL
    UNION
SELECT 'fire_names' AS table_name, count(*) as count FROM capstone.fire_names where fire_name_id IS NULL
    UNION
SELECT 'fire_sizes' AS table_name, count(*) as count FROM capstone.fire_sizes where fire_size_id IS NULL
    UNION
SELECT 'locations' AS table_name, count(*) as count FROM capstone.locations where location_id IS NULL    
    UNION
SELECT 'owners' AS table_name, count(*) as count FROM capstone.owners where owner_id IS NULL 
    UNION
SELECT 'dates' AS table_name, count(*) as count FROM capstone.dates where date_dim_id IS NULL
    UNION
SELECT 'weather_types' AS table_name, count(*) as count FROM capstone.weather_types where weather_type_id IS NULL
    UNION
SELECT 'weather_fact' AS table_name, count(*) as count FROM capstone.weather_fact where weather_outlier_id IS NULL
    UNION
SELECT 'fires_fact' AS table_name, count(*) as count FROM capstone.fires_fact where fire_id IS NULL
""", conn_athena)

,table_name,count
0,sources,0
1,fires_fact,0
2,weather_types,0
3,weather_fact,0
4,fire_causes,0
5,locations,0
6,owners,0
7,fire_sizes,0
8,dates,0
9,fire_names,0


In [30]:
pd.read_sql("""
SELECT 'fires_fact' AS table_name, count(*) AS count FROM capstone.fires_fact
UNION
SELECT 'weather_fact' AS table_name, count(*) AS count FROM capstone.weather_fact
""", conn_athena)

,table_name,count
0,fires_fact,1880465
1,weather_fact,1352248
